### Importando as bibliotecas necessárias

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import locale
import openpyxl
import re

#### Carregando o arquivo CSV em um DataFrame
#### Neste passo, carregaremos o arquivo CSV com os dados dos anúncios em um DataFrame do pandas
#### Certifique-se de ter o arquivo 'ANÁLISE-DE-MÉTRICAS-POR-ANÚNCIO-DESDE-DE-JANEIRO.csv' no mesmo diretório deste notebook

In [2]:
dados_anuncios = pd.read_csv('ANÁLISE-DE-MÉTRICAS-POR-ANÚNCIO-DESDE-DE-JANEIRO.csv')

In [3]:
dados_anuncios

,Nome do anúncio,Alcance,Impressões,Frequência,Valor usado (BRL),Configuração de atribuição,Tipo de resultado,Resultados,Custo por resultado,CPM (custo por 1.000 impressões),...,CPC (custo por clique no link),Cliques (todos),Engajamento com a Página,Reproduções de 25% do vídeo,Reproduções de 50% do vídeo,Reproduções de 75% do vídeo,Reproduções de 95% do vídeo,Reproduções de 100% do vídeo,Início dos relatórios,Término dos relatórios
0,AD 01 - CARROSSEL FEED - 05/06/2024,167158,409103,2.447403,562.85,Clique de 7 dias ou visualização de 1 dia,Alcance,167158.0,3.367174,1.375815,...,1.710790,625,381.0,NaN,NaN,NaN,NaN,NaN,2024-01-01,2024-10-30
1,AD 01 - CARROSSEL - 18/06/2024,120917,253308,2.094892,640.82,Clique de 7 dias ou visualização de 1 dia,Alcance,120917.0,5.299668,2.529806,...,2.926119,481,262.0,NaN,NaN,NaN,NaN,NaN,2024-01-01,2024-10-30
2,AD 01 - VIDEO - 16/09/2024,95498,178060,1.864542,340.81,Clique de 7 dias ou visualização de 1 dia,Alcance,95498.0,3.568766,1.914018,...,170.405000,159,6643.0,4553.0,1840.0,1163.0,890.0,839.0,2024-01-01,2024-10-30
3,AD 01 - VIDEO - 01/10/2024,81964,159631,1.947575,297.95,Clique de 7 dias ou visualização de 1 dia,Alcance,81964.0,3.635133,1.866492,...,148.975000,124,4955.0,2660.0,1178.0,742.0,206.0,189.0,2024-01-01,2024-10-30
4,AD 01 - STORY - 16/10/2024,59677,74129,1.242170,119.48,Clique de 7 dias ou visualização de 1 dia,Alcance,59677.0,2.002111,1.611785,...,NaN,35,5.0,NaN,NaN,NaN,NaN,NaN,2024-01-01,2024-10-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,AD 02 - IMAGEM ÚNICA - 17 | 08,0,0,0.000000,0.00,NaN,NaN,NaN,NaN,NaN,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,2024-01-01,2024-10-30
71,AD 02 - IMAGEM ÚNICA - 18 | 08,0,0,0.000000,0.00,NaN,NaN,NaN,NaN,NaN,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,2024-01-01,2024-10-30
72,AD 02 - CARROSSEL 2 - 21 | 12,0,0,0.000000,0.00,NaN,NaN,NaN,NaN,NaN,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,2024-01-01,2024-10-30
73,AD 01 - IMAGEM ÚNICA - 14 | 07,0,0,0.000000,0.00,NaN,NaN,NaN,NaN,NaN,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,2024-01-01,2024-10-30


### Removendo linhas duplicadas
### Vamos remover duplicatas, pois elas podem distorcer as métricas e análises dos dados de anúncios

In [4]:
# Identifica e exibe as linhas duplicadas antes de removê-las
duplicatas = dados_anuncios[dados_anuncios.duplicated()]

# Verifica se existem duplicatas e exibe uma mensagem
if not duplicatas.empty:
    print("Linhas duplicadas encontradas e que serão removidas:")
    print(duplicatas)
else:
    print("Nenhuma linha duplicada encontrada.")

# Remove as linhas duplicadas
dados_anuncios = dados_anuncios.drop_duplicates()


Linhas duplicadas encontradas e que serão removidas:
                   Nome do anúncio  Alcance  Impressões  Frequência  \
44   AD 01 - CARROSSEL 1 - 18 | 04        0           0         0.0   
47     AD 01 - CARROSSEL - 17 | 02        0           0         0.0   
50    AD 01 IMAGEM ÚNICA - 06 | 10        0           0         0.0   
51  AD 02 - IMAGEM ÚNICA - 18 | 08        0           0         0.0   
54       AD 03 CARROSSEL - 06 | 10        0           0         0.0   
55  AD 01 - IMAGEM ÚNICA - 18 | 08        0           0         0.0   
58   AD 01 - CARROSSEL 1 - 18 | 04        0           0         0.0   
59    AD 02 IMAGEM ÚNICA - 06 | 10        0           0         0.0   
60   AD 02 - CARROSSEL 1 - 18 | 04        0           0         0.0   
61     AD 03 - CARROSSEL - 17 | 08        0           0         0.0   
62     AD 03 - CARROSSEL - 18 | 08        0           0         0.0   
66  AD 01 - IMAGEM ÚNICA - 17 | 08        0           0         0.0   
67  AD 01 - IMAGEM ÚNICA

### Removendo colunas vazias
### Aqui, removeremos colunas que estejam completamente vazias, pois não contribuem para a análise

In [5]:
dados_anuncios = dados_anuncios.dropna(axis=1, how='all')

In [6]:
# Função para extrair a data
def extrair_data(nome_anuncio):
    match = re.search(r'\d{2}/\d{2}/\d{4}', nome_anuncio)
    return match.group(0) if match else None

# Aplicando a função na coluna 'Nome do anúncio' e criando a nova coluna 'Data de criação'
dados_anuncios['Data de criação'] = dados_anuncios['Nome do anúncio'].apply(extrair_data)

# Exibindo o DataFrame atualizado
dados_anuncios.head()


,Nome do anúncio,Alcance,Impressões,Frequência,Valor usado (BRL),Configuração de atribuição,Tipo de resultado,Resultados,Custo por resultado,CPM (custo por 1.000 impressões),...,Cliques (todos),Engajamento com a Página,Reproduções de 25% do vídeo,Reproduções de 50% do vídeo,Reproduções de 75% do vídeo,Reproduções de 95% do vídeo,Reproduções de 100% do vídeo,Início dos relatórios,Término dos relatórios,Data de criação
0,AD 01 - CARROSSEL FEED - 05/06/2024,167158,409103,2.447403,562.85,Clique de 7 dias ou visualização de 1 dia,Alcance,167158.0,3.367174,1.375815,...,625,381.0,NaN,NaN,NaN,NaN,NaN,2024-01-01,2024-10-30,05/06/2024
1,AD 01 - CARROSSEL - 18/06/2024,120917,253308,2.094892,640.82,Clique de 7 dias ou visualização de 1 dia,Alcance,120917.0,5.299668,2.529806,...,481,262.0,NaN,NaN,NaN,NaN,NaN,2024-01-01,2024-10-30,18/06/2024
2,AD 01 - VIDEO - 16/09/2024,95498,178060,1.864542,340.81,Clique de 7 dias ou visualização de 1 dia,Alcance,95498.0,3.568766,1.914018,...,159,6643.0,4553.0,1840.0,1163.0,890.0,839.0,2024-01-01,2024-10-30,16/09/2024
3,AD 01 - VIDEO - 01/10/2024,81964,159631,1.947575,297.95,Clique de 7 dias ou visualização de 1 dia,Alcance,81964.0,3.635133,1.866492,...,124,4955.0,2660.0,1178.0,742.0,206.0,189.0,2024-01-01,2024-10-30,01/10/2024
4,AD 01 - STORY - 16/10/2024,59677,74129,1.242170,119.48,Clique de 7 dias ou visualização de 1 dia,Alcance,59677.0,2.002111,1.611785,...,35,5.0,NaN,NaN,NaN,NaN,NaN,2024-01-01,2024-10-30,16/10/2024


### Exibindo os tipos de dados de cada coluna
### Essa inspeção é fundamental para verificar a consistência de tipos e ajustar a análise posteriormente, se necessário

In [7]:
dados_anuncios.dtypes

Nome do anúncio                              object
Alcance                                       int64
Impressões                                    int64
Frequência                                  float64
Valor usado (BRL)                           float64
Configuração de atribuição                   object
Tipo de resultado                            object
Resultados                                  float64
Custo por resultado                         float64
CPM (custo por 1.000 impressões)            float64
CTR (todos)                                 float64
Classificação de qualidade                   object
Classificação da taxa de engajamento         object
Conversas por mensagem iniciadas            float64
Custo por conversa por mensagem iniciada    float64
CPC (custo por clique no link)              float64
Cliques (todos)                               int64
Engajamento com a Página                    float64
Reproduções de 25% do vídeo                 float64
Reproduções 

# Ajuste de tipos das colunas

In [8]:
# Configura o locale para o Brasil
locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')

'pt_BR.UTF-8'

In [ ]:

# Função para formatar valores em moeda brasileira
def formatar_moeda(valor):
    return locale.currency(valor, grouping=True) if pd.notnull(valor) else valor

# Função para converter datas para o formato padrão e exibi-las no formato brasileiro
def converter_data(data):
    for fmt in ('%d/%m/%Y', '%Y-%m-%d'):
        try:
            data_formatada = pd.to_datetime(data, format=fmt)
            return data_formatada.strftime('%d/%m/%Y')  # Retorna em formato brasileiro
        except ValueError:
            continue
    return pd.NaT  # Retorna NaT se nenhum formato for adequado

# Aplicando as correções de tipos nas colunas conforme solicitado
dados_anuncios = dados_anuncios.astype({
    'Nome do anúncio': 'string',
    'Alcance': 'int64',
    'Impressões': 'int64',
    'Frequência': 'float64',
    'Configuração de atribuição': 'category',
    'Tipo de resultado': 'category',
    'Resultados': 'float64',
    'Classificação de qualidade': 'category',
    'Classificação da taxa de engajamento': 'category',
    'Cliques (todos)': 'int64',
    'Engajamento com a Página': 'float64',
    'Reproduções de 25% do vídeo': 'float64',
    'Reproduções de 50% do vídeo': 'float64',
    'Reproduções de 75% do vídeo': 'float64',
    'Reproduções de 95% do vídeo': 'float64',
    'Reproduções de 100% do vídeo': 'float64'
})

# Aplicando a função de formatação de moeda nas colunas de valores monetários
for coluna_monetaria in ['Valor usado (BRL)', 'Custo por resultado', 'CPM (custo por 1.000 impressões)', 'CPC (custo por clique no link)']:
    dados_anuncios[coluna_monetaria] = dados_anuncios[coluna_monetaria].apply(formatar_moeda)

# Convertendo datas para o formato brasileiro com a função converter_data
dados_anuncios['Início dos relatórios'] = dados_anuncios['Início dos relatórios'].apply(converter_data)
dados_anuncios['Término dos relatórios'] = dados_anuncios['Término dos relatórios'].apply(converter_data)

# Verificando datas nulas após a conversão
print("Datas nulas em 'Início dos relatórios':")
print(dados_anuncios[dados_anuncios['Início dos relatórios'].isnull()])

print("Datas nulas em 'Término dos relatórios':")
print(dados_anuncios[dados_anuncios['Término dos relatórios'].isnull()])

# Convertendo CTR para float64
dados_anuncios['CTR (todos)'] = pd.to_numeric(dados_anuncios['CTR (todos)'], errors='coerce')


Datas nulas em 'Início dos relatórios':
Empty DataFrame
Columns: [Nome do anúncio, Alcance, Impressões, Frequência, Valor usado (BRL), Configuração de atribuição, Tipo de resultado, Resultados, Custo por resultado, CPM (custo por 1.000 impressões), CTR (todos), Classificação de qualidade, Classificação da taxa de engajamento, Conversas por mensagem iniciadas, Custo por conversa por mensagem iniciada, CPC (custo por clique no link), Cliques (todos), Engajamento com a Página, Reproduções de 25% do vídeo, Reproduções de 50% do vídeo, Reproduções de 75% do vídeo, Reproduções de 95% do vídeo, Reproduções de 100% do vídeo, Início dos relatórios, Término dos relatórios, Data de criação]
Index: []

[0 rows x 26 columns]
Datas nulas em 'Término dos relatórios':
Empty DataFrame
Columns: [Nome do anúncio, Alcance, Impressões, Frequência, Valor usado (BRL), Configuração de atribuição, Tipo de resultado, Resultados, Custo por resultado, CPM (custo por 1.000 impressões), CTR (todos), Classificação d

In [10]:
# Exibir o dataframe atualizado e verificar os tipos das colunas
dados_anuncios.dtypes

Nome do anúncio                               object
Alcance                                        int64
Impressões                                     int64
Frequência                                   float64
Valor usado (BRL)                             object
Configuração de atribuição                  category
Tipo de resultado                           category
Resultados                                   float64
Custo por resultado                           object
CPM (custo por 1.000 impressões)              object
CTR (todos)                                  float64
Classificação de qualidade                  category
Classificação da taxa de engajamento        category
Conversas por mensagem iniciadas             float64
Custo por conversa por mensagem iniciada     float64
CPC (custo por clique no link)                object
Cliques (todos)                                int64
Engajamento com a Página                     float64
Reproduções de 25% do vídeo                  f

In [11]:
# Identificar as linhas onde o valor de 'Alcance' é igual a zero
linhas_excluir = dados_anuncios[dados_anuncios['Alcance'] == 0]

# Extrair os nomes dos anúncios que serão excluídos
anuncios_excluidos = linhas_excluir['Nome do anúncio'].tolist()

# Imprimir os nomes dos anúncios que serão excluídos de forma mais bonita
print("Anúncios excluídos:")
for i, anuncio in enumerate(anuncios_excluidos, start=1):
    print(f"{i}. {anuncio}")

# Excluir as linhas onde 'Alcance' é igual a zero do DataFrame original
dados_anuncios = dados_anuncios[dados_anuncios['Alcance'] != 0]

# Resetar o índice após a exclusão
dados_anuncios.reset_index(drop=True, inplace=True)


Anúncios excluídos:
1. AD 04 - VÍDEO - 18 | 04
2. AD 01 - STORIES - 21 | 12
3. AD 03 - CARROSSEL - 17 | 08
4. AD 04 - CARROSSEL 4  - 21 | 12
5. AD 01 - CARROSSEL 1 - 18 | 04
6. AD 02 - CARROSSEL 1 - 18 | 04
7. AD 03 - CARROSSEL 3  - 21 | 12
8. AD 01 IMAGEM ÚNICA - 06 | 10
9. AD 03 - CARROSSEL - 18 | 08
10. AD 03 - IMAGEM ÚNICA - 18 | 04
11. AD 01 - CARROSSEL - 17 | 02
12. AD 01 - IMAGEM ÚNICA - 18 | 08
13. AD 03 - VÍDEO - 18 | 04
14. AD 03 CARROSSEL - 06 | 10
15. AD 02 IMAGEM ÚNICA - 06 | 10
16. AD 02 - IMAGEM ÚNICA - 18 | 08
17. AD 02 - IMAGEM ÚNICA - 17 | 08
18. AD 01 - CARROSSEL 1 - 21 | 12
19. AD 02 - IMAGEM ÚNICA - 14 | 07
20. AD 02 - VÍDEO - 18 | 04
21. AD 01 - VÍDEO - 30 | 06
22. AD 01 - VÍDEO - 18 | 04
23. AD 01 - IMAGEM ÚNICA - 17 | 08
24. AD 02 - CARROSSEL 2 - 18 | 04
25. AD 02 - CARROSSEL 2  - 21 | 12
26. AD 01 - IMAGEM ÚNICA - 14 | 07
27. AD 01 - ESPECIAL - 21 | 12


In [12]:
# Exibir o DataFrame atualizado para ver o resultado
dados_anuncios

,Nome do anúncio,Alcance,Impressões,Frequência,Valor usado (BRL),Configuração de atribuição,Tipo de resultado,Resultados,Custo por resultado,CPM (custo por 1.000 impressões),...,Cliques (todos),Engajamento com a Página,Reproduções de 25% do vídeo,Reproduções de 50% do vídeo,Reproduções de 75% do vídeo,Reproduções de 95% do vídeo,Reproduções de 100% do vídeo,Início dos relatórios,Término dos relatórios,Data de criação
0,AD 01 - CARROSSEL FEED - 05/06/2024,167158,409103,2.447403,"R$ 562,85",Clique de 7 dias ou visualização de 1 dia,Alcance,167158.0,"R$ 3,37","R$ 1,38",...,625,381.0,NaN,NaN,NaN,NaN,NaN,01/01/2024,30/10/2024,05/06/2024
1,AD 01 - CARROSSEL - 18/06/2024,120917,253308,2.094892,"R$ 640,82",Clique de 7 dias ou visualização de 1 dia,Alcance,120917.0,"R$ 5,30","R$ 2,53",...,481,262.0,NaN,NaN,NaN,NaN,NaN,01/01/2024,30/10/2024,18/06/2024
2,AD 01 - VIDEO - 16/09/2024,95498,178060,1.864542,"R$ 340,81",Clique de 7 dias ou visualização de 1 dia,Alcance,95498.0,"R$ 3,57","R$ 1,91",...,159,6643.0,4553.0,1840.0,1163.0,890.0,839.0,01/01/2024,30/10/2024,16/09/2024
3,AD 01 - VIDEO - 01/10/2024,81964,159631,1.947575,"R$ 297,95",Clique de 7 dias ou visualização de 1 dia,Alcance,81964.0,"R$ 3,64","R$ 1,87",...,124,4955.0,2660.0,1178.0,742.0,206.0,189.0,01/01/2024,30/10/2024,01/10/2024
4,AD 01 - STORY - 16/10/2024,59677,74129,1.242170,"R$ 119,48",Clique de 7 dias ou visualização de 1 dia,Alcance,59677.0,"R$ 2,00","R$ 1,61",...,35,5.0,NaN,NaN,NaN,NaN,NaN,01/01/2024,30/10/2024,16/10/2024
5,AD 01 - VIDEO - 16/09/2024,50748,63471,1.250709,"R$ 188,24",Clique de 7 dias ou visualização de 1 dia,Alcance,50748.0,"R$ 3,71","R$ 2,97",...,103,2441.0,1774.0,746.0,453.0,374.0,353.0,01/01/2024,30/10/2024,16/09/2024
6,AD 01 - VÍDEO STORIES - 04/04/24,42284,172197,4.072391,"R$ 641,46",Clique de 7 dias ou visualização de 1 dia,Visualização contínua de 2 segundos do vídeo,25807.0,"R$ 0,02","R$ 3,73",...,109,17748.0,29900.0,14375.0,9803.0,7925.0,7760.0,01/01/2024,30/10/2024,None
7,AD 01 - VIDEO - 16/09/2024,34031,42545,1.250184,"R$ 114,90",Clique de 7 dias ou visualização de 1 dia,Alcance,34031.0,"R$ 3,38","R$ 2,70",...,49,1314.0,723.0,330.0,219.0,60.0,56.0,01/01/2024,30/10/2024,16/09/2024
8,AD 01 - STORY - 05/06/2024,33903,65337,1.927175,"R$ 90,97",Clique de 7 dias ou visualização de 1 dia,Alcance,33903.0,"R$ 2,68","R$ 1,39",...,62,34.0,NaN,NaN,NaN,NaN,NaN,01/01/2024,30/10/2024,05/06/2024
9,AD 02 - VIDEO - 16/09/2024,15838,17583,1.110178,"R$ 50,11",Clique de 7 dias ou visualização de 1 dia,Alcance,15838.0,"R$ 3,16","R$ 2,85",...,21,1388.0,2073.0,584.0,335.0,251.0,240.0,01/01/2024,30/10/2024,16/09/2024


In [13]:
dados_anuncios.columns

Index(['Nome do anúncio', 'Alcance', 'Impressões', 'Frequência',
       'Valor usado (BRL)', 'Configuração de atribuição', 'Tipo de resultado',
       'Resultados', 'Custo por resultado', 'CPM (custo por 1.000 impressões)',
       'CTR (todos)', 'Classificação de qualidade',
       'Classificação da taxa de engajamento',
       'Conversas por mensagem iniciadas',
       'Custo por conversa por mensagem iniciada',
       'CPC (custo por clique no link)', 'Cliques (todos)',
       'Engajamento com a Página', 'Reproduções de 25% do vídeo',
       'Reproduções de 50% do vídeo', 'Reproduções de 75% do vídeo',
       'Reproduções de 95% do vídeo', 'Reproduções de 100% do vídeo',
       'Início dos relatórios', 'Término dos relatórios', 'Data de criação'],
      dtype='object')

In [14]:
dados_anuncios.to_csv("miro_dados_desde_janeiro.csv", index=False)